This script quantifies non-additive effects as in PMID 25897392.

In [1]:
import pandas, numpy

In [2]:
import matplotlib, matplotlib.pyplot
matplotlib.rcParams.update({'font.size':20, 'font.family':'sans-serif', 'xtick.labelsize':30, 'ytick.labelsize':30, 'figure.figsize':(16, 9), 'axes.labelsize':40})

# 0. user-defined variables

In [3]:
expression_file = '/home/adrian/projects/nautholsvik/results/tpm/DESeq2_TPM_values.tsv'
significance_dir = '/home/adrian/projects/nautholsvik/results/DEGs_DESeq2/'

In [4]:
genotypes = ['siCTRL', 'siMITF']
treatments = ['wo_IFN', 'with_IFN']

# 1. read and manipulate expression data

In [5]:
expression_with_replicates = pandas.read_csv(expression_file, sep='\t', index_col=0)

In [6]:
plotting_df = pandas.DataFrame()
replicate_labels = expression_with_replicates.columns

for genotype in genotypes:
    for treatment in treatments:
        new_label = genotype + '_' + treatment + '_tpm'
        sub = [label for label in replicate_labels if genotype in label and treatment in label]
        df = expression_with_replicates[sub]
        plotting_df[new_label] = df.median(axis=1)

In [7]:
plotting_df['average_expression_tpm'] = plotting_df.mean(axis=1)
plotting_df['max_expression_tpm'] = plotting_df.max(axis=1)

print(plotting_df.shape)
plotting_df.head()

(40173, 6)


,siCTRL_wo_IFN_tpm,siCTRL_with_IFN_tpm,siMITF_wo_IFN_tpm,siMITF_with_IFN_tpm,average_expression_tpm,max_expression_tpm
ENSG00000000003,24.864996,22.810767,23.996147,23.638828,23.827684,24.864996
ENSG00000000005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,148.830008,143.782572,159.665244,156.673309,152.237783,159.665244
ENSG00000000457,6.434201,9.625513,9.285772,11.832608,9.294523,11.832608
ENSG00000000460,42.553495,32.939327,33.307114,22.274693,32.768657,42.553495


# 2. add fold-changes of each independent effect

In [8]:
plotting_df['A1A0_log2fc'] = numpy.log2(plotting_df['siCTRL_with_IFN_tpm']+1) - numpy.log2(plotting_df['siCTRL_wo_IFN_tpm']+1)
plotting_df['B0A0_log2fc'] = numpy.log2(plotting_df['siMITF_wo_IFN_tpm']+1) - numpy.log2(plotting_df['siCTRL_wo_IFN_tpm']+1)

plotting_df.head()

,siCTRL_wo_IFN_tpm,siCTRL_with_IFN_tpm,siMITF_wo_IFN_tpm,siMITF_with_IFN_tpm,average_expression_tpm,max_expression_tpm,A1A0_log2fc,B0A0_log2fc
ENSG00000000003,24.864996,22.810767,23.996147,23.638828,23.827684,24.864996,-0.119387,-0.049295
ENSG00000000005,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000000419,148.830008,143.782572,159.665244,156.673309,152.237783,159.665244,-0.049439,0.100731
ENSG00000000457,6.434201,9.625513,9.285772,11.832608,9.294523,11.832608,0.515283,0.468401
ENSG00000000460,42.553495,32.939327,33.307114,22.274693,32.768657,42.553495,-0.359831,-0.344281


# 3. read fold changes and P values from DESeq2

In [9]:
# read signifcances
df = pandas.read_csv(significance_dir+'A0_A1_transition.tsv', sep='\t', index_col=0)
plotting_df['P_A1A0'] = df['padj']

df = pandas.read_csv(significance_dir+'A0_B0_transition.tsv', sep='\t', index_col=0)
plotting_df['P_B0A0'] = df['padj']

df = pandas.read_csv(significance_dir+'interaction.tsv', sep='\t', index_col=0)
plotting_df['P_interaction'] = df['padj']

# 4. remove element whose top expression is below 2 TPM

In [10]:
plotting_df.fillna(1, inplace=True)

In [11]:
# remove low-expression genes: less than two tpm
print(plotting_df.shape)
plotting_df.drop(plotting_df[plotting_df['max_expression_tpm'] < 2].index, inplace=True)
print(plotting_df.shape)

(40173, 11)
(12742, 11)


In [12]:
plotting_df.head()

,siCTRL_wo_IFN_tpm,siCTRL_with_IFN_tpm,siMITF_wo_IFN_tpm,siMITF_with_IFN_tpm,average_expression_tpm,max_expression_tpm,A1A0_log2fc,B0A0_log2fc,P_A1A0,P_B0A0,P_interaction
ENSG00000000003,24.864996,22.810767,23.996147,23.638828,23.827684,24.864996,-0.119387,-0.049295,1.000000e+00,0.047093,1.000000e+00
ENSG00000000419,148.830008,143.782572,159.665244,156.673309,152.237783,159.665244,-0.049439,0.100731,1.000000e+00,1.000000,1.000000e+00
ENSG00000000457,6.434201,9.625513,9.285772,11.832608,9.294523,11.832608,0.515283,0.468401,4.729447e-06,0.000422,1.000000e+00
ENSG00000000460,42.553495,32.939327,33.307114,22.274693,32.768657,42.553495,-0.359831,-0.344281,5.019661e-03,0.000003,1.000000e+00
ENSG00000000971,0.362546,2.480156,0.127168,9.427765,3.099409,9.427765,1.352847,-0.273603,4.909581e-46,0.003251,3.700648e-16


In [13]:
# these numbers is to see the effect of removing low-expressed genes, it is not represtantive of the final set of genes, specially for the interaction
# originally, at the significance level we had A1A0: 4,414; B0A0: 9,116 and interaction: 700
print(sum(plotting_df['P_A1A0'] < 0.1))
print(sum(plotting_df['P_B0A0'] < 0.1))
print(sum(plotting_df['P_interaction'] < 0.1))

3929
7633
631


In [14]:
# check if MITF is A0 to B0 transition
print('ENSG00000187098' in plotting_df[plotting_df['P_B0A0'] < 0.1].index)

# check if PD-L1 is a gene with significant interaction
print('ENSG00000120217' in plotting_df[plotting_df['P_interaction'] < 0.1].index)

True
True


# 5. add a column of the expected expression given both factors

In [15]:
# add expected expression
deltaA = plotting_df['siCTRL_with_IFN_tpm'] - plotting_df['siCTRL_wo_IFN_tpm']
deltaB = plotting_df['siMITF_wo_IFN_tpm'] - plotting_df['siCTRL_wo_IFN_tpm']
expected = plotting_df['siCTRL_wo_IFN_tpm'] + deltaA + deltaB
plotting_df['expected'] = expected 
plotting_df[plotting_df['expected'] < 0] = 0 # avoid predicting negative expression

plotting_df.head()

,siCTRL_wo_IFN_tpm,siCTRL_with_IFN_tpm,siMITF_wo_IFN_tpm,siMITF_with_IFN_tpm,average_expression_tpm,max_expression_tpm,A1A0_log2fc,B0A0_log2fc,P_A1A0,P_B0A0,P_interaction,expected
ENSG00000000003,24.864996,22.810767,23.996147,23.638828,23.827684,24.864996,-0.119387,-0.049295,1.000000e+00,0.047093,1.000000e+00,21.941919
ENSG00000000419,148.830008,143.782572,159.665244,156.673309,152.237783,159.665244,-0.049439,0.100731,1.000000e+00,1.000000,1.000000e+00,154.617808
ENSG00000000457,6.434201,9.625513,9.285772,11.832608,9.294523,11.832608,0.515283,0.468401,4.729447e-06,0.000422,1.000000e+00,12.477084
ENSG00000000460,42.553495,32.939327,33.307114,22.274693,32.768657,42.553495,-0.359831,-0.344281,5.019661e-03,0.000003,1.000000e+00,23.692946
ENSG00000000971,0.362546,2.480156,0.127168,9.427765,3.099409,9.427765,1.352847,-0.273603,4.909581e-46,0.003251,3.700648e-16,2.244778


In [17]:
# add log2FC observed / expected expression
non_additive = numpy.log2(plotting_df['siMITF_with_IFN_tpm']+1) - numpy.log2(plotting_df['expected']+1)
plotting_df['log2FC_observed_expected'] = non_additive

plotting_df.head()

,siCTRL_wo_IFN_tpm,siCTRL_with_IFN_tpm,siMITF_wo_IFN_tpm,siMITF_with_IFN_tpm,average_expression_tpm,max_expression_tpm,A1A0_log2fc,B0A0_log2fc,P_A1A0,P_B0A0,P_interaction,expected,log2FC_observed_expected
ENSG00000000003,24.864996,22.810767,23.996147,23.638828,23.827684,24.864996,-0.119387,-0.049295,1.000000e+00,0.047093,1.000000e+00,21.941919,0.102948
ENSG00000000419,148.830008,143.782572,159.665244,156.673309,152.237783,159.665244,-0.049439,0.100731,1.000000e+00,1.000000,1.000000e+00,154.617808,0.018931
ENSG00000000457,6.434201,9.625513,9.285772,11.832608,9.294523,11.832608,0.515283,0.468401,4.729447e-06,0.000422,1.000000e+00,12.477084,-0.070694
ENSG00000000460,42.553495,32.939327,33.307114,22.274693,32.768657,42.553495,-0.359831,-0.344281,5.019661e-03,0.000003,1.000000e+00,23.692946,-0.085337
ENSG00000000971,0.362546,2.480156,0.127168,9.427765,3.099409,9.427765,1.352847,-0.273603,4.909581e-46,0.003251,3.700648e-16,2.244778,1.684238


In [21]:
sum(numpy.abs(plotting_df['log2FC_observed_expected']) > 1)

198

# 6. learn type of non-additive effect

# 3. plot

In [18]:
# select genes that will be plotted
rules = (plotting_df['P_A1A0'] < 0.1) | (plotting_df['P_B1B0'] < 0.1) | (plotting_df['P_interaction'] < 0.1)
scatter_df = plotting_df[rules]
scatter_df.shape

KeyError: 'P_B1B0'

In [ ]:
scatter_df